In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Sequential,Model,load_model
from tensorflow.keras.layers import Input, Activation, Flatten, Dense, Dropout,Reshape
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras import activations
import time
import math

In [ ]:
from tensorflow.keras.datasets import cifar10
(img_train, label_train), (img_test, label_test) = cifar10.load_data()

170500096/170498071 [==============================] - 4s 0us/step


In [ ]:
x_train = img_train.astype('float32') / 255
x_test = img_test.astype('float32') / 255

In [ ]:
y_train = to_categorical(label_train)
y_test = to_categorical(label_test)

In [ ]:
model = load_model('drive/My Drive/Do_an-NCKH/model_Conv2D.h5')

In [ ]:
model_poly.save('drive/My Drive/Do_an-NCKH/model_poly.h5')

#Client

In [ ]:
x = x_test[0:3]
x.shape

(3, 32, 32, 3)

In [ ]:
stride, kernel, padding = 1, 3, 0

In [ ]:
leghtX = x.shape[0]
heightX = x.shape[1]
weightX = x.shape[2]
deptX = x.shape[3]

In [ ]:
H = math.floor((heightX - kernel + 2*padding) / stride) + 1
W = math.floor((weightX - kernel + 2*padding) / stride) + 1
L = kernel*kernel*deptX

In [ ]:
X = np.zeros((leghtX, H, W, L, 2, 2))
k = np.array([[np.random.rand(),np.random.rand()],
              [np.random.rand(),np.random.rand()]])
kd = np.linalg.inv(k)

In [ ]:
for xi in range(leghtX):
  for hi in range(0, heightX, stride):
    for wi in range(0, weightX, stride):
      x_ = x[xi,hi:hi+kernel,wi:wi+kernel,:].reshape(-1)
      for i_ in np.arange(L):
        X_ = np.array([[x_[i_], np.random.rand()],
                       [np.random.rand(), np.random.rand()]])
        X[xi, hi, wi, i_] = k.dot(X_)
        # X[xi, hi, wi, i_] = X_
      if wi + kernel == weightX:
        break
    if hi + kernel == heightX:
      break

In [ ]:
X.shape

(3, 30, 30, 27, 2, 2)

#Server

In [ ]:
wx = np.zeros((3,30,30,96,2,2))

In [ ]:
model_poly.layers[0].output_shape

(None, 30, 30, 96)

In [ ]:
w0, b0 = model_poly.layers[0].get_weights()
model_ = Sequential()
for layer in model_poly.layers[1:]:
  model_.add(layer)
model_.build(input_shape = model_poly.layers[0].output_shape)

In [ ]:
w0.shape

(3, 3, 3, 96)

#Xem1

In [ ]:
listw = []
for d in np.arange(w0.shape[-1]):
  w_ = w0[:,:,:,d].reshape(-1)
  listw.append(w_)
w = np.array(listw)

In [ ]:
w.shape

(96, 27)

In [ ]:
for xi in np.arange(X.shape[0]):
  for hi in np.arange(X.shape[1]):
    for wi in np.arange(X.shape[2]):
      for w_ in np.arange(w.shape[0]):
        sumR = 0
        for ri in np.arange(X.shape[3]):
          r = w[w_,ri]*X[xi, hi, wi, ri]
          sumR += r
        wx[xi, hi, wi, w_] = sumR 

In [ ]:
x00 = model_.predict(wx[:,:,:,:,0,0] )
x01 = model_.predict(wx[:,:,:,:,0,1] )
x10 = model_.predict(wx[:,:,:,:,1,0] )
x11 = model_.predict(wx[:,:,:,:,1,1] )
xr = np.zeros((3,10,2,2))
for i in range(3):
  for j in range(3):
    xr_ = np.array([[x00[i,j], x01[i,j]],
                   [x10[i,j], x11[i,j]]])
    xr__ = kd.dot(xr_)
    xr[i,j] = xr__ 
lst = []
for ii in range(3):
  lst.append(xr[ii,:,0,0].argmax())

In [ ]:
print(lst)

[2, 1, 3]


In [ ]:
y_uncrypt = model_poly.predict(x)
lst_uncrypt = []
for jj in range(y_uncrypt.shape[0]):
  lst_uncrypt.append(y_uncrypt[jj].argmax()) 
print(lst_uncrypt)

[3, 8, 2]


Thuật toán: Đối với CNN: Client phải có Stride, padding, kernel của model, tính toán output. Rồi gửi về Server ở ví dụ này X có shape (5,30,30,27,2,2). Phía Server tính toán sắp xếp lại shape của weight ở đây là (32,27), rồi sau đó tính toán như code trên để output giống như input lúc nhận. Sau đó cho vào model dự đoán từng giá trị của ma trận (2,2). Gửi về client. Client giải mã bằng K^(-1)

In [ ]:
p = 0.09
0.12*(p**2) + 0.5*p + 0.409

0.454972

In [ ]:
def polyfit(px):
  return 0.1524*(px**2) + 0.5*px + 0.409

In [ ]:
img_size = 32

In [ ]:
model_poly = Sequential()
img_input = Input(shape=(img_size,img_size,3))
 
model_poly.add(Conv2D(96, (3,3), padding='valid', input_shape = img_input.shape[1:]))
# model_poly.add(Activation(polyfit))
# model_poly.add(AveragePooling2D(pool_size=(2,2), padding='valid'))
model_poly.add(BatchNormalization())
model_poly.add(Conv2D(96, (3,3),padding='valid'))
model_poly.add(BatchNormalization())
model_poly.add(Conv2D(96, (3,3),padding='valid'))
model_poly.add(Activation(polyfit))
model_poly.add(BatchNormalization())

model_poly.add(Conv2D(192, (3,3), padding='valid', input_shape = img_input.shape[1:]))
model_poly.add(BatchNormalization())
model_poly.add(Conv2D(192, (3,3), padding='valid', input_shape = img_input.shape[1:]))
model_poly.add(BatchNormalization())
model_poly.add(Activation(polyfit))
model_poly.add(Conv2D(192, (3,3), padding='valid', input_shape = img_input.shape[1:]))
model_poly.add(BatchNormalization())
model_poly.add(Conv2D(192, (3,3), padding='valid', input_shape = img_input.shape[1:]))
model_poly.add(BatchNormalization())
model_poly.add(Conv2D(192, (3,3), padding='valid', input_shape = img_input.shape[1:]))
model_poly.add(BatchNormalization())
model_poly.add(Activation(polyfit))

model_poly.add(Flatten())
model_poly.add(Dense(10))
# model_poly.add(GlobalAveragePooling2D())
model_poly.add(Activation(activations.sigmoid))

In [ ]:
model_relu = Sequential()
img_input = Input(shape=(img_size,img_size,3))
 
model_relu.add(Conv2D(32, (3,3), padding='valid', input_shape = img_input.shape[1:]))
model_relu.add(Activation(activations.relu))
model_relu.add(AveragePooling2D(pool_size=(2,2), padding='valid'))
 
model_relu.add(Conv2D(64, (3,3),padding='same'))
model_relu.add(Activation(activations.relu))
model_relu.add(AveragePooling2D(pool_size=(2,2), padding='same'))
 
model_relu.add(Conv2D(128, (3,3),padding='same'))
model_relu.add(Activation(activations.relu))
model_relu.add(AveragePooling2D(pool_size=(2,2), padding='same'))
model_relu.add(Flatten())
model_relu.add(Dense(256))
model_relu.add(Activation(activations.relu))
model_relu.add(Dense(64))
model_relu.add(Activation(activations.relu))
model_relu.add(Dense(32))
model_relu.add(Activation(activations.relu))
model_relu.add(Dense(10))
model_relu.add(Activation(activations.sigmoid))

In [ ]:
sgdx = SGD(lr=0.005, decay =1e-6, momentum=0.9, nesterov=True)
ntrain = 60
otp = Adam(learning_rate=1e-4)

In [ ]:
s = time.time()
model_poly.compile(optimizer=otp,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
H_new = model_poly.fit(x_train, y_train, epochs=10)
print('time: ',time.time() - s)

Epoch 1/10
1563/1563 [==============================] - 50s 32ms/step - loss: 0.1494 - accuracy: 0.9477
Epoch 2/10
1563/1563 [==============================] - 48s 31ms/step - loss: 0.1414 - accuracy: 0.9513
Epoch 3/10
1563/1563 [==============================] - 49s 31ms/step - loss: 0.1371 - accuracy: 0.9532
Epoch 4/10
1563/1563 [==============================] - 49s 31ms/step - loss: 0.1449 - accuracy: 0.9505
Epoch 5/10
1563/1563 [==============================] - 49s 31ms/step - loss: 0.1303 - accuracy: 0.9559
Epoch 6/10
1563/1563 [==============================] - 49s 31ms/step - loss: 0.1342 - accuracy: 0.9546
Epoch 7/10
1563/1563 [==============================] - 49s 31ms/step - loss: 0.1219 - accuracy: 0.9590
Epoch 8/10
1563/1563 [==============================] - 49s 31ms/step - loss: 0.1182 - accuracy: 0.9614
Epoch 9/10
1563/1563 [==============================] - 49s 31ms/step - loss: 0.1275 - accuracy: 0.9594
Epoch 10/10
1563/1563 [==============================] - 49s 31m

In [ ]:
s = time.time()
model_relu.compile(optimizer=otp,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
H_new = model_relu.fit(x_train, y_train,
                    epochs=50)
print('time: ',time.time() - s)

In [ ]:
 
model_relu.evaluate(x_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 2.2745 - accuracy: 0.7215


[2.2745187282562256, 0.7214999794960022]

#Test